Issue https://community.home-assistant.io/t/hue-motion-sensors-remotes-custom-component/27176/209

Hi, I have now checked the hue api and the motion sensor not showing up in ha is showing under api/sensors.
Here is the complete responce from the api showing all 8 motion sensors. https://pastebin.com/nRTJDRUM2
The strange thing is that it is sensor number 6 that is not showing up.

In [1]:
import hue_sensors as hs

In [2]:
hs.__version__

1.2

In [3]:
filename = "hue_error_pastebin.json" # From pastebin

In [4]:
response = hs.load_json_file(filename)

In [5]:
data = hs.parse_hue_api_response(response)

In [6]:
data

{'RWL_06-02': {'battery': 100,
  'last_updated': ['2018-01-28', '11:28:35'],
  'model': 'RWL',
  'name': 'badrummet dimmer switch',
  'state': '4_click'},
 'RWL_6c-02': {'battery': 100,
  'last_updated': ['2018-01-25', '18:59:01'],
  'model': 'RWL',
  'name': 'Vardagsrummet dimmer switch 2',
  'state': '1_click'},
 'RWL_86-02': {'battery': 100,
  'last_updated': ['2018-01-28', '13:25:25'],
  'model': 'RWL',
  'name': 'köket dimmer switch',
  'state': '4_click'},
 'RWL_9e-02': {'battery': 100,
  'last_updated': ['2018-01-25', '18:58:53'],
  'model': 'RWL',
  'name': 'Vardagsrummet dimmer switch 1',
  'state': '4_click'},
 'RWL_b8-02': {'battery': 100,
  'last_updated': ['2018-01-28', '13:25:26'],
  'model': 'RWL',
  'name': 'Baksidan dimmer switch',
  'state': '4_click'},
 'RWL_b9-02': {'battery': 100,
  'last_updated': ['2018-01-28', '13:25:25'],
  'model': 'RWL',
  'name': 'Teverummet dimmer switch',
  'state': '4_click'},
 'RWL_c5-02': {'battery': 100,
  'last_updated': ['2018-01-25'

In [7]:
keys = list(data.keys())
print(len(keys))
keys

14


['RWL_b8-02',
 'RWL_c5-02',
 'RWL_6c-02',
 'RWL_9e-02',
 'RWL_b9-02',
 'SML_1e-02',
 'SML_73-02',
 'SML_fa-02',
 'SML_4a-02',
 'SML_2e-02',
 'SML_9e-02',
 'SML_2a-02',
 'RWL_86-02',
 'RWL_06-02']

7 remotes, 7 motion sensors.

## Issue fix
As proposed by @sevenofnine-jni all characters of the unique_id can be used except for the final four where 0400 = light, 0402 = temp and 0406 is motion.

In [21]:
uniqueid = "00:17:88:01:02:12:dc:d0-02-0402"

In [22]:
key = uniqueid[:-5]

In [23]:
key

'00:17:88:01:02:12:dc:d0-02'

Lets update the parsing function and try that

In [24]:
def parse_hue_api_response(response):
    """Take in the Hue API json response."""
    data_dict = {}    # The list of sensors, referenced by their hue_id.

    # Loop over all keys (1,2 etc) to identify sensors and get data.
    for key in response.keys():
        sensor = response[key]
        modelid = sensor['modelid'][0:3]
        if modelid in ['RWL', 'SML', 'ZGP']:
            _key = modelid + '_' + sensor['uniqueid'][:-5]  # Updated key

            if modelid == 'RWL':
                data_dict[_key] = hs.parse_rwl(sensor)
            elif modelid == 'ZGP':
                data_dict[_key] = hs.parse_zgp(sensor)
            elif modelid == 'SML':
                if _key not in data_dict.keys():
                    data_dict[_key] = hs.parse_sml(sensor)
                else:
                    data_dict[_key].update(hs.parse_sml(sensor))

        elif sensor['modelid'] == 'HA_GEOFENCE':
            data_dict['Geofence'] = hs.parse_geofence(sensor)
    return data_dict

In [25]:
data_new = parse_hue_api_response(response)

In [26]:
data_new

{'RWL_00:17:88:01:02:0e:9d:9e-02': {'battery': 100,
  'last_updated': ['2018-01-25', '18:58:53'],
  'model': 'RWL',
  'name': 'Vardagsrummet dimmer switch 1',
  'state': '4_click'},
 'RWL_00:17:88:01:02:0e:9d:b8-02': {'battery': 100,
  'last_updated': ['2018-01-28', '13:25:26'],
  'model': 'RWL',
  'name': 'Baksidan dimmer switch',
  'state': '4_click'},
 'RWL_00:17:88:01:02:0e:9e:c5-02': {'battery': 100,
  'last_updated': ['2018-01-25', '19:00:03'],
  'model': 'RWL',
  'name': 'Framsidan dimmer switch',
  'state': '1_click'},
 'RWL_00:17:88:01:02:0f:f3:6c-02': {'battery': 100,
  'last_updated': ['2018-01-25', '18:59:01'],
  'model': 'RWL',
  'name': 'Vardagsrummet dimmer switch 2',
  'state': '1_click'},
 'RWL_00:17:88:01:02:0f:fe:86-02': {'battery': 100,
  'last_updated': ['2018-01-28', '13:25:25'],
  'model': 'RWL',
  'name': 'köket dimmer switch',
  'state': '4_click'},
 'RWL_00:17:88:01:02:0f:fe:b9-02': {'battery': 100,
  'last_updated': ['2018-01-28', '13:25:25'],
  'model': 'RWL

In [27]:
keys = list(data_new.keys())
print(len(keys))
keys

15


['RWL_00:17:88:01:02:0e:9d:b8-02',
 'RWL_00:17:88:01:02:0e:9e:c5-02',
 'RWL_00:17:88:01:02:0f:f3:6c-02',
 'RWL_00:17:88:01:02:0e:9d:9e-02',
 'RWL_00:17:88:01:02:0f:fe:b9-02',
 'SML_00:17:88:01:02:10:7d:1e-02',
 'SML_00:17:88:01:02:12:37:73-02',
 'SML_00:17:88:01:02:13:e6:fa-02',
 'SML_00:17:88:01:02:13:db:4a-02',
 'SML_00:17:88:01:02:13:a1:2e-02',
 'SML_00:17:88:01:02:13:96:9e-02',
 'SML_00:17:88:01:02:13:bb:2a-02',
 'SML_00:17:88:01:02:13:db:2e-02',
 'RWL_00:17:88:01:02:0f:fe:86-02',
 'RWL_00:17:88:01:02:c1:13:06-02']

7 remotes and 8 motion sensors.

In [ ]:
Error fixed.